In [1]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [1]:
import ee
import datetime
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../../')

### Step_0_Define_Basic_Parameters

In [3]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2023,3)][-1:]

In [4]:
year_range

[('2020-01-01', '2022-12-31')]

##### Adjust below code according to your reserach area

In [5]:
# define region EN/CN dict
region_name_dict = dict(zip(['东北', '华北', '华东',  '西北', '西南', '中南'],
                            ['dongbei', 'huabei', 'huadong', 'xibei', 'xinan', 'zhongnan']))

region_name_dict

{'东北': 'dongbei',
 '华北': 'huabei',
 '华东': 'huadong',
 '西北': 'xibei',
 '西南': 'xinan',
 '中南': 'zhongnan'}

In [6]:
target_regions = ['东北', '华北', '华东',  '中南'] # '西北', '西南'

In [7]:
# define the asset path for exporting
asset_path = 'users/cnbuilt001/Cloud_free'

### Step_1: Create Landsat cloud-free images

In [22]:
for region_cn in target_regions:

    # get the area shp for export
    Target_area = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
                    .filterMetadata('NAME1','equals',region_cn)
    
    # get the English name of reigon
    region_en = region_name_dict[region_cn]

    for span in year_range:
        
        # find the end year to determine the 
        start = span[0]
        end   = span[1]       
        split = '-'
        span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'
    
        # define the name for exporting
        export_name = f'Landsat_cloud_free_{span_name}_{region_en}'
             
        # find the right Landsat RAW data set
        if int(end.split(split)[0]) <= 2010:
            Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1")         
        elif int(end.split(split)[0]) <= 2013:
            Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1")            
        else:
            Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1")
            
        # Create a function to compute the min of the input mask to 
        # mask out  pixels of incomplete band numbers
        def min_mask (img):
            return img.updateMask(img.mask().reduce('min'))
        
        # create the cloud-free img using SimpleComposite
        Filtered_img = Landsat_img.filterBounds(Target_area)\
                                  .filterDate(start,end)\
                                  .map(lambda img: min_mask(img))
        
        cloud_free = ee.Algorithms.Landsat.simpleComposite(Filtered_img)\
                                  .clip(Target_area)
            
        # export
        task = ee.batch.Export.image.toAsset(   image       = cloud_free,
                                                description = f'Exporting {export_name}',
                                                assetId     = f'{asset_path}/{export_name}',
                                                region      = Target_area.geometry(),
                                                scale       = 30,
                                                maxPixels   = int(1e13))
        task.start()
        
        # print out the process
        print(f'Exporting {export_name}')

Exporting Landsat_cloud_free_2020_2022_dongbei_2020_2022
Exporting Landsat_cloud_free_2020_2022_huabei_2020_2022
Exporting Landsat_cloud_free_2020_2022_huadong_2020_2022
Exporting Landsat_cloud_free_2020_2022_zhongnan_2020_2022


##### Add img to map

In [23]:
one_img = cloud_free

In [25]:
# show the map for validation
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(120.3482, 33.0212,8)

# add the img
Map.addLayer(one_img,{'min':0,'max':100,'bands':['B4','B3','B2']},'Landsat_cloud_free')


# show the img
Map

### Step_2: Create Sentinel cloud-free Image

In [7]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask  = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask)\
                 .select("B.*")\
                 .copyProperties(image, ["system:time_start"])

In [12]:
# here composite Sentinel-2 multispectrum image of year [2015-2017] for classification of [2014-2016]
#            and Sentinel-2 multispectrum image of year [2018-2019] for classification of [2017-2019]
# because composition of S2 image at year [2014-2016] are not fully cloud-free

Sentinel_year_range = [("2018-01-01","2019-12-31"),("2015-01-01","2017-12-31")]

Sentinel_img =  [ee.ImageCollection("COPERNICUS/S2")\
                      .filterBounds(North_China_Plain)\
                      .filterDate(*S_t)\
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                      .map(lambda x: maskS2clouds(x))\
                      .median()
                      .clip(North_China_Plain.geometry()) for S_t in Sentinel_year_range] 

# add 'Sentinel' to each band name of Sentinel img
Sentinel_img = [img.rename([f'Sentinel_{band}' for band in  img.bandNames().getInfo()]) for img in Sentinel_img]